# Tutorial 1: Basic Use Cases

In this tutorial, we are going to see how we can use **RadPrompter** for simplified and reproducible LLM prompting. We will cover more advanced capabilities in the next tutorials.

RadPrompter uses three simple components:

1. **Prompt**: The `Prompt` is the core message-passing recipe between the user and the model.
2. **Client**: The `Client` is responsible for contacting various LLM servers.
3. **Engine**: The `Engine` is the core functionality that ties the other two components together and coordinates running a job.

## Prompt

Let's start with the `Prompt`. We first have to import it from the **RadPrompter** package.

In [3]:
from radprompter import Prompt

Prompts can be initiated by a toml file. Let's read `01_Basic-Usecase.toml` to see its structure:

In [4]:
with open("./01_Basic-Usecase.toml", "r") as f:
    lines = f.readlines()
print("".join(lines))

[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"


[CONSTRUCTOR]
system = "You are an experienced radiologist that help users extract infromation from radiology reports."
user = """"Does the following report indicate a normal or abnormal finding?
{{report}}

Just reply with "normal" or "abnormal" to indicate your answer, without any additional information.
"""


As you can see, the minimum requirements for a prompt file are to have a `[METADATA]` and a `[CONSTRUCTOR]` section. The metadata is used for version tracking and verbose communication. The constructor is used to craft the LLM chat structure.

In this example, we specify a `system` attribute which will be the system prompt for the LLM. We also have a `user` attribute which contains the user's request. You might notice the `{{report}}` tag being enclosed in double curly brackets. This denotes a **placeholder** item that can be replaced with your actual report. Note that this curly bracket placeholder notation can be mixed with any arbitrary variable name that you like, and "report" is just an example. We will learn more about this as we move forward.

Let's now create our prompt:

In [6]:
prompt = Prompt("./01_Basic-Usecase.toml")
prompt

As you can see, each prompt comes with a tabular visualization for easier interpretation. The placeholders will be in a light orange color, and the assistant's response will replace the blue `[... response ...]` tags.

## Client

RadPrompter supports several LLM clients out of the box, including:

- `OpenAIClient`: For accessing OpenAI's models
- `vLLMClient`: For accessing open-source LLMs like Llama hosted using the [vLLM package](https://vllm.ai/).
- `OllamaClient`: For accesing [Ollama](https://ollama.com/) open source models.

To instantiate a client, you need to provide the following:

1. `model` [Required]: The name of the model to use (e.g., "gpt-3.5-turbo" for OpenAI,  "meta-llama/Meta-Llama-3-8B-Instruct" for vLLM and "phi3" for Ollama).
2. `base_url` [Optional]: The URL of the REST API endpoint.
3. `api_key` [Optional]): Your API key for the service. If not provided, the client will attempt to read it from an environment variable (in case of `OpenAIClient`) or set that to "EMPTY" (`vLLMClient` and `OllamaClient`).

Here's an example of instantiating a vLLMClient:

In [7]:
from radprompter import vLLMClient

client = vLLMClient(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    base_url="http://localhost:9999",
    api_key="EMPTY",
)

Now that we have set up our client, we need to coonect everything together using the RadPrompter **Engine**.

## Engine

The `RadPrompter` class is the engine that ties everything together. To use it, we need to provide:

1. `client`: The LLM client to use 
2. `prompt`: The prompt object
3. `output_file`: The path to save the results to (must be a .csv file)

Let's instantiate the engine:

In [8]:
from radprompter import RadPrompter

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_1.csv"
)

To run the engine, we pass it a list of dictionaries. Each dictionary should contain keys matching the placeholders in the prompt. Any additional keys will be included in the output file. 

So let's create a list of our inputs:

In [15]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

In [10]:
engine(reports)

Processing items: 0it [00:00, ?it/s]


ZeroDivisionError: float division by zero